## Deduplicating data

In this notebook, we deduplicate data using the [Dedupe library](https://dedupe.readthedocs.io/en/latest/), which utilizes a shallow neural network to learn from a small training exercise.

If you are interested in building your own parser, the same folks have created the [Parserator](https://github.com/datamade/parserator) which you can use to extract text features and train your own text extraction (hooray! less brittle than regex!)

In [6]:
pip install dedupe


  Using cached dedupe-2.0.23-cp311-cp311-win_amd64.whl (96 kB)
  Using cached affinegap-1.12.tar.gz (33 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached categorical_distance-1.9-py3-none-any.whl (3.3 kB)
  Using cached dedupe_variable_datetime-1.0.0-py3-none-any.whl (3.9 kB)
  Using cached DoubleMetaphone-1.1.tar.gz (34 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached highered-0.2.1-py2.py3-none-

  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [56 lines of output]
  C:\Users\admin\AppData\Local\Temp\pip-build-env-4fs4gpqo\overlay\Lib\site-packages\Cython\Compiler\Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: C:\Users\admin\AppData\Local\Temp\pip-install-scob0k16\pylbfgs_c90dbaa5a59c47c7ba8a0d3b0650eb89\lbfgs\_lowlevel.pyx
    tree = Parsing.p_module(s, pxd, full_module_name)
  
  Error compiling Cython file:
  ------------------------------------------------------------
  ...
  
          x_a = aligned_copy(x0.ravel())
  
          try:
              callback_data = (f, progress, x0.shape, args)
              r = lbfgs(n, x_a, fx_final, call_eval,
                                          ^
  ------------------------------------------------------------
  
  lbfgs\_lowlevel.pyx:395:40: Cannot assign type '

In [11]:
import pandas as pd
import dedpupe
import os

ModuleNotFoundError: No module named 'dedpupe'

In [7]:
customers = pd.read_csv('customer_data_duped.csv', 
                        encoding='utf-8')

## Checking Data Quality

In [9]:
customers.head()

,name,job,company,street_address,city,state,email,user_name
0,Patricia Schaefer,"Programmer, systems",Estrada-Best,398 Paul Drive,Christianview,Delaware,lambdavid@gmail.com,ndavidson
1,Olivie Dubois,Ingénieur recherche et développement en agroal...,Moreno,rue Lucas Benard,Saint Anastasie-les-Bains,AR,berthelotjacqueline@mahe.fr,manonallain
2,Mary Davies-Kirk,Public affairs consultant,Baker Ltd,Flat 3\nPugh mews,Stanleyfurt,ZA,middletonconor@hotmail.com,colemanmichael
3,Miroslawa Eckbauer,Dispensing optician,Ladeck GmbH,Mijo-Lübs-Straße 12,Neubrandenburg,Berlin,sophia01@yahoo.de,romanjunitz
4,Richard Bauer,"Accountant, chartered certified",Hoffman-Rocha,6541 Rodriguez Wall,Carlosmouth,Texas,tross@jensen-ware.org,adam78


In [ ]:
customers.dtypes

In [ ]:
for col in customers.columns:
    print(col, customers[col].isnull().sum())

## Setting up Dedupe

In [ ]:
variables = [
    {'field': 'name', 'type': 'String'},
    {'field': 'job', 'type': 'String'},
    {'field': 'company', 'type': 'String'},  
    {'field': 'street_address','type': 'String'},
    {'field': 'city','type': 'String'},
    {'field': 'state', 'type': 'String', 'has_missing': True},
    {'field': 'email', 'type': 'String', 'has_missing': True},
    {'field': 'user_name', 'type': 'String'},
]

deduper = dedupe.Dedupe(variables)

In [ ]:
deduper

In [ ]:
customers.shape

In [ ]:
deduper.sample(customers.T.to_dict(), 500)

Note: If you receive an error like this:

```/usr/local/lib/python2.7/site-packages/dedupe/sampling.py:39: UserWarning: 250 blocked samples were requested, but only able to sample 249
  % (sample_size, len(blocked_sample)))
```

you can continue (some were selected), or use the suggested number (^ here it would be 249)

#### Either use training file (uncomment) or resume active training below

In [ ]:
training_file = '../data/ignore-dedupe-training.json'
#if os.path.exists(training_file):
#    with open(training_file, 'rb') as f:
#        deduper.readTraining(f)

In [ ]:
dedupe.consoleLabel(deduper)

In [ ]:
deduper.train()

In [ ]:
with open(training_file, 'w') as tf:
    deduper.writeTraining(tf)

In [ ]:
dupes = deduper.match(customers.T.to_dict())

In [ ]:
dupes

In [ ]:
dupes[2]

In [ ]:
customers.iloc[[741,1107]]

### Exercise: Flag duplicates by adding 2 extra columns, one for confidence score and one for duplicate_ids

In [ ]:
# %load ../solutions/dedupe.py


In [ ]:
customers[customers.confidence.notnull() == True].head()